# RAG Chatbot using Ollama for LLM & Embeddings

In [7]:
import ollama
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

## Download ollama in the system first

- visit [ollama offical website] (https://ollama.com/download)
- download the model you want (in this case, mistral)
- run on terminal ollama run mixtral
- run on terminal ollama serve (checking for ollama is running or not)

In [8]:
# initilize ollama library

llm = Ollama(model="mistral")

# check ollama is working or not

llm.invoke("Tell me a joke")

" Sure! Here's one:\n\nWhy don't some numbers play hide and seek?\n\nBecause they're easy to find. Especially zero, because you can always find it hiding behind every other number!"

### Getting output as in string format

In [9]:
# use langchain sting output parser

parser = StrOutputParser()

chain = llm | parser
chain.invoke("Who owns microsoft?")

' Microsoft is a publicly-traded company (symbol: MSFT), meaning it is owned by many individual and institutional shareholders. The largest individual shareholder as of March 2021 is Bill Gates, who co-founded the company but has sold some of his shares over time. However, he no longer holds a majority stake in Microsoft.'

### create chat prompt template for chatbot 

In [10]:


template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(
            context='''RAG is a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG).''' , 
            question="What is full form of RAG?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: RAG is a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG).\n\nQuestion: What is full form of RAG?\n'

### combine prompt template to model and output parser

In [11]:
chain = prompt | llm | parser

# check output is given or not
chain.invoke({
    "context": "RAG is a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG).",
    "question": "What is full form of RAG?"
})

' I don\'t have access to external databases or knowledge bases to determine the full form of an acronym that isn\'t commonly known. In this case, "RAG" does not appear to be an established acronym with a widely recognized meaning, so it doesn\'t have a standard full form in the way that, for example, "HTML" has a full form of "Hypertext Markup Language". The context provided suggests that RAG stands for "Retrieval-Augmented Generation," which is its function as described.'

## Creating RAG model for Chatbot

In [12]:
# load pdf from device

loader = PyPDFLoader("Who Moved My Cheese_.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 0}, page_content="Who Moved My Cheese? Summary\nBy  Spencer Johnson\nChange is the only certainty in life, so why do we avoid and even fear it?\nWe know that change is inevitable, yet we lack effective strategies to\ncome to terms with it. What if there was a roadmap to help us better\nnavigate change?\nWho Moved My Cheese  is a light-hearted fable that offers deep insights into\nadapting to change in multiple scenarios. In a simple story of the\ninevitability of change, two mice and two little people navigate living in a\nmaze and maintaining their cheese supplies. The characters reveal that\nwe can either accept change, or complain about it.\nThis summary takes a brief look at Dr. Johnson's story. It tackles the\ncomplexity of how we react to change, as a means to inspire us to be\ncourageous in the face of it. Growth often emerges because of adversity,\nand sometimes unexpected scenarios arise when we least expect the

### Split the documents into chunks

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pages)

### Initialize Embeeding model of Ollama

In [14]:
embeddings = (OllamaEmbeddings(model='mistral'))  

text = "This is a test document."

In [15]:
# checking embedding model is working or not

query_result = embeddings.embed_query(text)
print(f"Embedding length: {len(query_result)}")
query_result[:5]

Embedding length: 4096


[1.8355729579925537,
 -2.2049717903137207,
 0.10405337810516357,
 -10.774480819702148,
 1.3713029623031616]

### Store Vector Embeeding into local database using FAISS

In [16]:

db = FAISS.from_documents(documents=documents, embedding=embeddings)

In [17]:
# save and load embedding in device so we save time

db.save_local("faiss_index") #after first run comment this and load the saved embeedings
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [18]:
# create retriver

retriver = db.as_retriever()

## Create a chain

In [19]:
# load runnable passthorugh

setup = RunnableParallel(context=retriver, question=RunnablePassthrough())
setup.invoke("Who is author of book?")

{'context': [Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 3}, page_content='monitoring the initial cheese scenario. They remained action-focused,\nrather than becoming caught in over-analysis and stale resentments.'),
  Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 6}, page_content="keep things simple. The two little people's complex brains and human\nemotions, tend to complicate things. Whatever parts of us we choose to\nembrace, or choose to ignore, we all have something in common. All of us\nhave the universal need to find our way around the maze, and succeed in\nchanging times.\nIn Conclusion"),
  Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 1}, page_content="might see them. Eventually, Haw discovers a new station with an\nabundance of cheese, and he reunites with Sniff and Scurry. They have\nbeen there for some time, enjoying the fruits of their labor.\nOne of the book's central messages is that Haw's notes on the wall ser

In [20]:
# combine all the code

chain = (
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [21]:
# check working or not

chain.invoke("What is names of mice")

' The names of the mice in "Who Moved My Cheese" are Sniff and Scurry.'

In [22]:
chain.invoke("can you tell me learning of this book?")

' The learning from the book "Who Moved My Cheese" revolves around adapting to change and embracing it as a constant aspect of life. The story emphasizes the importance of staying action-focused instead of getting caught in over-analysis or stale resentments, keeping things simple, and navigating the maze (representing life) with the understanding that all of us have the universal need to find our way around and succeed in changing times.\n\nOne of the central messages is that everyone encounters obstacles and struggles on their journey of change, but we should learn from the characters in the story and remember that fear is what stops us from moving into the unknown. To overcome this, it is suggested that one should ask themselves what they would do if fear weren\'t holding them back.\n\nIn conclusion, the learning from "Who Moved My Cheese" encourages readers to embrace change, adapt quickly, learn from their experiences, and take risks in order to find success and happiness in life\

In [23]:
chain.invoke("what is mean by keep things simple?")

' In the context provided, "keep things simple" means avoiding unnecessary complexity and focusing on the essential aspects of a situation. This concept is emphasized as a way to navigate through change more effectively, as complex thoughts and emotions can often complicate matters, making it difficult to adapt in changing times.'